# Recall of cause metrics

In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
SOCKSHOP_DATASET_ID = "9n6mf"

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["font.size"] = 7
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['axes.linewidth'] = 1.0
plt.rcParams['axes.grid'] = True

In [3]:
import sys
sys.path.append('../')

In [4]:
from meltria import loader
from eval import validation

In [12]:
metrics_files = !find "/datasets/argowf-chaos-{SOCKSHOP_DATASET_ID}/" -type f -name "*.json"
records = loader.load_dataset(metrics_files, target_metric_types={
        "containers": True,
        "services": True,
        "nodes": True,
        "middlewares": True,
    },
    num_datapoints=4*45, # 45min
)

In [13]:
well_injected_records = validation.find_records_detected_anomalies_of_sli(records, faulty_datapoints=4*5, sli_index=0)
well_injected_records = validation.find_records_detected_anomalies_of_cause_metrics(well_injected_records, faulty_datapoints=4*5)

In [8]:
from eval.tsdr import sweep_tsdr_for_recall

In [40]:
score_df = sweep_tsdr_for_recall(
    records=well_injected_records, 
    list_of_tsdr_options=[
        {
            "enable_unireducer": True,
            "enable_multireducer": True,
            "step1_enable_smoother": True,
            "step1_smoother_window_size": 4,
            "step1_method_name": "residual_integral",
            "step1_residual_integral_threshold": 20,
            "step1_residual_integral_change_start_point": False,
            "step1_residual_integral_change_start_point_n_sigma": 3,
            "step2_enable_smoother": True,
            "step2_smoother_window_size": 4,
            "step2_dbscan_min_pts": 1,
            "step2_dbscan_dist_type": "pearsonr",  # 'pearsonr' or 'sbd'
            "step2_dbscan_algorithm": "dbscan",  # 'dbscan' or 'hdbscan'
            "step2_clustering_series_type": "raw",  # 'raw', 'anomaly_score' or 'binary_anomaly_score'
            "step2_clustering_choice_method": "medoid",  # 'medoid' or 'maxsum'
        },
    ],
    use_manually_selected_metrics=[False],
)

In [43]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    cols = score_df.loc[:, score_df.columns.str.contains("recall|n_", regex=True)].columns.tolist()
    display(score_df[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_type"])[cols].agg(["mean", "max", "min"]))
    display(score_df.groupby(["chaos_comp"])[cols].agg(["mean", "max", "min"]))
    display(score_df.set_index(["chaos_type", "chaos_comp", "chaos_case_num"]).sort_index())

,filtered/only_mandatory_recall,filtered/recall,anomalous/only_mandatory_recall,anomalous/recall,reduced/only_mandatory_recall,reduced/recall
mean,0.819113,0.660265,0.818791,0.657619,0.554747,0.451214
max,1.000000,0.925926,1.000000,0.925926,1.000000,0.800000
min,0.142857,0.125000,0.142857,0.125000,0.071429,0.074074


filtered/only_mandatory_recall                 \
                                              mean  max       min   
chaos_type                                                          
pod-cpu-hog                               0.845703  1.0  0.375000   
pod-memory-hog                            0.808364  1.0  0.142857   
pod-network-latency                       0.808316  1.0  0.153846   

                    filtered/recall                      \
                               mean       max       min   
chaos_type                                                
pod-cpu-hog                0.722137  0.809524  0.571429   
pod-memory-hog             0.586706  0.880000  0.125000   
pod-network-latency        0.690254  0.925926  0.148148   

                    anomalous/only_mandatory_recall                 \
                                               mean  max       min   
chaos_type                                                           
pod-cpu-hog                                0.845703  1.0  0.375000   
pod-memory-hog                             0.807514  1.0  0.142857   
pod-network-latency                        0.808316  1.0  0.153846   

                    anomalous/recall                      \
                                mean       max       min   
chaos_type                                                 
pod-cpu-hog                 0.720184  0.809524  0.571429   
pod-memory-hog              0.581200  0.880000  0.125000   
pod-network-latency         0.690254  0.925926  0.148148   

                    reduced/only_mandatory_recall                   \
                                             mean    max       min   
chaos_type                                                           
pod-cpu-hog                              0.544922  0.875  0.125000   
pod-memory-hog                           0.530976  0.800  0.071429   
pod-network-latency                      0.590229  1.000  0.076923   

                    reduced/recall                      
                              mean       max       min  
chaos_type                                              
pod-cpu-hog               0.464041  0.619048  0.315789  
pod-memory-hog            0.399023  0.708333  0.093750  
pod-network-latency       0.499363  0.800000  0.074074

filtered/only_mandatory_recall                filtered/recall  \
                                       mean  max       min            mean   
chaos_comp                                                                   
carts                              0.543956  1.0  0.153846        0.518263   
carts-db                           0.706169  1.0  0.375000        0.624552   
catalogue                          1.000000  1.0  1.000000        0.738095   
catalogue-db                       1.000000  1.0  1.000000        0.738889   
orders                             0.617882  1.0  0.142857        0.573609   
orders-db                          0.708988  1.0  0.375000        0.616411   
payment                            1.000000  1.0  1.000000        0.723333   
user                               1.000000  1.0  1.000000        0.738889   
user-db                            0.714286  1.0  0.375000        0.625444   

                                 anomalous/only_mandatory_recall       \
                   max       min                            mean  max   
chaos_comp                                                              
carts         0.925926  0.148148                        0.542170  1.0   
carts-db      0.909091  0.240000                        0.706169  1.0   
catalogue     0.800000  0.666667                        1.000000  1.0   
catalogue-db  0.800000  0.666667                        1.000000  1.0   
orders        0.925926  0.125000                        0.616259  1.0   
orders-db     0.909091  0.240000                        0.708988  1.0   
payment       0.800000  0.666667                        1.000000  1.0   
user          0.800000  0.666667                        1.000000  1.0   
user-db       0.909091  0.250000                        0.714286  1.0   

                       anomalous/recall                      \
                   min             mean       max       min   
chaos_comp                                                    
carts         0.153846         0.516700  0.925926  0.148148   
carts-db      0.375000         0.610266  0.909091  0.240000   
catalogue     1.000000         0.738095  0.800000  0.666667   
catalogue-db  1.000000         0.738889  0.800000  0.666667   
orders        0.142857         0.572188  0.925926  0.125000   
orders-db     0.375000         0.616411  0.909091  0.240000   
payment       1.000000         0.723333  0.800000  0.666667   
user          1.000000         0.734722  0.800000  0.666667   
user-db       0.375000         0.625444  0.909091  0.250000   

             reduced/only_mandatory_recall                     reduced/recall  \
                                      mean       max       min           mean   
chaos_comp                                                                      
carts                             0.366941  0.833333  0.071429       0.334436   
carts-db                          0.440097  0.750000  0.125000       0.407978   
catalogue                         0.672619  0.800000  0.400000       0.520337   
catalogue-db                      0.777778  1.000000  0.333333       0.584375   
orders                            0.399892  0.833333  0.107143       0.356158   
orders-db                         0.481198  0.875000  0.125000       0.417773   
payment                           0.728333  0.800000  0.600000       0.518194   
user                              0.645556  0.800000  0.333333       0.487500   
user-db                           0.452679  0.875000  0.125000       0.408840   

                                  
                   max       min  
chaos_comp                        
carts         0.592593  0.074074  
carts-db      0.680000  0.120000  
catalogue     0.600000  0.333333  
catalogue-db  0.800000  0.375000  
orders        0.593750  0.093750  
orders-db     0.636364  0.160000  
payment       0.600000  0.437500  
user          0.600000  0.333333  
user-db       0.708333  0.125000

filtered/only_mandatory_recall  \
chaos_type          chaos_comp   chaos_case_num                                   
pod-cpu-hog         carts        2                                     0.500000   
                                 2                                     1.000000   
                                 4                                     0.500000   
                                 4                                     1.000000   
                    carts-db     0                                     0.375000   
                                 0                                     1.000000   
                                 1                                     0.375000   
                                 1                                     1.000000   
                                 2                                     0.375000   
                                 2                                     1.000000   
                                 4                                     0.375000   
                                 4                                     1.000000   
                    catalogue    0                                     1.000000   
                                 0                                     1.000000   
                                 1                                     1.000000   
                                 1                                     1.000000   
                                 2                                     1.000000   
                                 2                                     1.000000   
                                 4                                     1.000000   
                                 4                                     1.000000   
                    catalogue-db 1                                     1.000000   
                                 1                                     1.000000   
                                 2                                     1.000000   
                                 2                                     1.000000   
                                 3                                     1.000000   
                                 3                                     1.000000   
                                 4                                     1.000000   
                                 4                                     1.000000   
                    orders       1                                     0.500000   
                                 1                                     1.000000   
                                 2                                     0.500000   
                                 2                                     1.000000   
                                 3                                     0.500000   
                                 3                                     1.000000   
                                 4                                     0.500000   
                                 4                                     1.000000   
                    orders-db    0                                     0.375000   
                                 0                                     1.000000   
                                 1                                     0.375000   
                                 1                                     1.000000   
                                 4                                     0.375000   
                                 4                                     1.000000   
                    payment      1                                     1.000000   
                                 1                                     1.000000   
                                 3                                     1.000000   
                                 3                                     1.000000   
                    user         0                               